In [1]:
from elasticsearch import Elasticsearch

In [3]:
INDEX_NAME = 'jawiki_articles'

es = Elasticsearch('http://localhost:9200/')

In [48]:
query_string = '嵐を呼ぶジャングル'

In [49]:
query = {
      "query":{
        "multi_match": {
          "query": query_string,
          "fields": ["title" , "text_morph", "text_ngram^0.1"]
        }
      }
    }

response = es.search(index=INDEX_NAME, body=query)

print("{} total hits.".format(response["hits"]["total"]["value"]))
for i, hit in enumerate(response["hits"]["hits"]):
    print(f'title: {hit["_source"]["title"]}, score: {hit["_score"]}')
    print(hit["_source"]["text_ngram"][:100])
    if i != 5:
      print()
    else:
      break

10000 total hits.
title: 嵐を呼ぶ男, score: 26.284885
『嵐を呼ぶ男』（あらしをよぶおとこ）は、1957年に公開された石原裕次郎主演の日本映画で石原の代表作の一つとして数えられている。北原三枝演じるヒロインのモデルは当時女性マネージャーの嚆矢として注目を集

title: クレヨンしんちゃん 嵐を呼ぶジャングル, score: 23.0592
『クレヨンしんちゃん 嵐を呼ぶジャングル』（クレヨンしんちゃん あらしをよぶジャングル）は、2000年4月22日に公開された『クレヨンしんちゃん』の劇場映画シリーズ第8作目。上映時間は88分。興行収入

title: 原勝徳, score: 23.050407
原 勝徳（はら かつのり、1月1日 - ）は、日本のアニメーター・キャラクターデザイナーである。 じゃんぐるじむ出身。現在は『クレヨンしんちゃん』を中心に活動しており、稀に元じゃんぐるじむのスタッフが

title: さよならありがとう, score: 22.754227
「さよならありがとう」は、2000年4月22日にリリースされた小林幸子のシングル（CODA-1867）。コーラスに音羽ゆりかご会が参加している。 2000年に公開された映画『クレヨンしんちゃん 嵐を呼

title: ARASHI No.1〜嵐は嵐を呼ぶ〜, score: 22.07739
『ARASHI No.1 (ICHIGOU) 〜嵐は嵐を呼ぶ〜』（アラシ イチゴウ あらしはあらしをよぶ）は、日本の男性アイドルグループ、嵐の1枚目のオリジナル・アルバム。2001年1月24日にポニー

title: 大熊昭, score: 21.848824
大熊 昭（おおくま あきら、1950年9月18日 - ）は、日本のアニメーションの音響監督。 1975年にオーディオ・プランニング・ユーに入社し、1981年、オーディオ・プランニング・ユー常務取締役、


In [36]:
query = {
  "query":{
    "function_score":{
      "query":{
        "multi_match": {
          "query": query_string,
          "fields": ["title" , "text_morph", "text_ngram^0.1"]
        }
      },
    "script_score":{
      "script":{
        "params":{
          "max_score":5
        },
        "source": "params.max_score * (1 - (1 / (10 * _score)))"
      }
    }
    }
  }
}

response = es.search(index=INDEX_NAME, body=query)

print("{} total hits.".format(response["hits"]["total"]["value"]))
for i, hit in enumerate(response["hits"]["hits"]):
    print(f'title: {hit["_source"]["title"]}, score: {hit["_score"]}')
    print(hit["_source"]["text_ngram"][:100])
    if i != 5:
      print()
    else:
      break

RequestError: RequestError(400, 'search_phase_execution_exception', 'script score function must not produce negative scores, but got: [-2.070795550138489]')